# Gemini API: Function calling with Python

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Function_calling.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


Function calling lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with. Function calling lets you use functions as tools in generative AI applications, and you can define more than one function within a single request.

This notebook provides code examples to help you get started.

In [ ]:
!pip install -U -q google-generativeai # Install the Python SDK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 933.2 kB/s eta 0:00:00


In [ ]:
import google.generativeai as genai

## Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Function calling basics

To use function calling, pass a list of functions to the `tools` parameter when creating a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel). The model uses the function name, docstring, parameters, and parameter type annotations to decide if it needs the function to best answer a prompt.

> Important: The SDK converts function parameter type annotations to a format the API understands (`glm.FunctionDeclaration`). The API only supports a limited selection of parameter types, and the Python SDK's automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`

In [ ]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a-b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[add, subtract, multiply, divide])

model

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7a99071b61d0>,
)

## Automatic function calling

Function calls naturally fit in to [multi-turn chats](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#multi-turn) as they capture a back and forth interaction between the user and model. The Python SDK's [`ChatSession`](https://ai.google.dev/api/python/google/generativeai/ChatSession) is a great interface for chats because handles the conversation history for you, and using the parameter `enable_automatic_function_calling` simplifies function calling even further:

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

With automatic function calling enabled, `ChatSession.send_message` automatically calls your function if the model asks it to.

In the following example, the result appears to simply be a text response containing the correct answer:

In [ ]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

'There are 2508 mittens in total.'

In [ ]:
57*44

2508

However, by examining the chat history, you can see the flow of the conversation and how function calls are integrated within it.

The `ChatSession.history` property stores a chronological record of the conversation between the user and the Gemini model. Each turn in the conversation is represented by a [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `ChatSession` automatically executes the function (due to `enable_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [ ]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'There are 2508 mittens in total.'}]
--------------------------------------------------------------------------------


In general the state diagram is:

<img src="https://ai.google.dev/images/tutorials/function_call_state_diagram.png" alt="The model can always reply with text, or a FunctionCall. Iff the model sends a FunctionCall the user must reply with a FunctionResponse" width=50%>

The model can respond with multiple function calls before returning a text response, and function calls come before the text response.

## Manual function calling

For more control, you can process [`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall) requests from the model yourself. This would be the case if:

- You use a `ChatSession` with the default `enable_automatic_function_calling=False`.
- You use `GenerativeModel.generate_content` (and manage the chat history yourself).

The following example is a rough equivalent of the [function calling single-turn curl sample](https://ai.google.dev/docs/function_calling#function-calling-single-turn-curl-sample) in Python. It uses functions that return (mock) movie playtime information, possibly from a hypothetical API:

In [ ]:
def find_movies(description: str, location: str = ''):
  """find movie titles currently playing in theaters based on any description, genre, title words, etc.

  Args:
      description: Any kind of description including category or genre, title words, attributes, etc.
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
  """
  return ['Barbie', 'Oppenheimer']

def find_theaters(location: str, movie: str = ''):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ['Googleplex 16', 'Android Theatre']

def get_showtimes(location:str, movie:str, theater:str, date:str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ['10:00', '11:00']

Use a dictionary to make looking up functions by name easier later on. You can also use it to pass the array of functions to the `tools` parameter of `GenerativeModel`.

In [ ]:
functions = {
    'find_movies': find_movies,
    'find_theaters': find_theaters,
    'get_showtimes': get_showtimes,
}

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=functions.values())

After using `generate_content()` to ask a question, the model requests a `function_call`:

In [ ]:
response = model.generate_content('Which theaters in Mountain View show the Barbie movie?')
response.candidates[0].content.parts

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
  }
}
]

Since this is not using a `ChatSession` with automatic function calling, you have to call the function yourself.

A very simple way to do this would be with `if` statements:

```python
if function_call.name == 'find_theaters':
  find_theaters(**function_call.args)
elif ...
```

However, since we made the `functions` dictionary earlier, we can simplify this to:

In [ ]:
def call_function(function_call, functions):
  function_name = function_call.name
  function_args = function_call.args
  return functions[function_name](**function_args)

part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
  result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']


Finally, pass the response plus the message history to the next `generate_content()` call to get a final text response from the model.

In [ ]:
import google.ai.generativelanguage as glm
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
s.update({'result': result})

# Update this after https://github.com/google/generative-ai-python/issues/243
function_response = glm.Part(
    function_response=glm.FunctionResponse(name='find_theaters', response=s))

# Build the message history
messages = [
    {'role':'user',
     'parts': ['Which theaters in Mountain View show the Barbie movie?.']},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': [function_response]}
]

# Generate the next response
response = model.generate_content(messages)
print(response.text)

Theaters showing Barbie in Mountain View, CA are: Googleplex 16, Android Theatre


## Next Steps

Useful API references:

- The [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) class
  - Its [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) method builds a [glm.GenerateContentRequest](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentRequest) behind the scenes.
    - The request's `.tools` field contains a list of 1 [glm.Tool](https://ai.google.dev/api/python/google/ai/generativelanguage/Tool) object.
    - The tool's `function_declarations` attribute contains a list of [FunctionDeclarations](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionDeclaration) objects.
- The [response](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) may contain a [glm.FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall), in `response.candidates[0].contents.parts[0]`.
- if `enable_automatic_function_calling` is set the [genai.ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) executes the call, and sends back the [glm.FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse).
- In response to a [FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall) the model always expects a [FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse).
- If you reply manually using [chat.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession#send_message) or [model.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) remember thart the API is stateless you have to send the whole conversation history (a list of [content](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) objects), not just the last one containing the `FunctionResponse`.